In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
bike_data = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv")

In [15]:
bike_data.head()



,Unnamed: 0,instant,dteday,season,hr,holiday,workingday,weathersit,hum,windspeed,temp_c,feels_like_c,casual,registered
0,0,1,1/1/11,1,0,0,0,1,0.81,0,3.28,3.0014,3,13
1,1,2,1/1/11,1,1,0,0,1,0.80,0,2.34,1.9982,8,32
2,2,3,1/1/11,1,2,0,0,1,0.80,0,2.34,1.9982,5,27
3,3,4,1/1/11,1,3,0,0,1,0.75,0,3.28,3.0014,3,10
4,4,5,1/1/11,1,4,0,0,1,0.75,0,3.28,3.0014,0,1
